In [1]:
import numpy as np
import pandas as pd 
import re 
from nltk.corpus import stopwords
from string import punctuation
import re
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

In [2]:
df=pd.read_csv('./IMDB_dataset/IMDB dataset.csv')

stop = set(stopwords.words('english'))
punctuation = list(punctuation)
stop.update(punctuation)

def remove_stopwords(text, stopwords_set):
    output = []
    for i in text.split():
        word = i.strip().lower()
        if word not in stopwords_set and word.isalpha():
            output.append(word)
    return " ".join(output)
    
def process_data(text):
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text()
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
    text = remove_stopwords(text, stop)
    return text

df['review']=df['review'].apply(process_data)

df.sentiment.replace("positive" , 1 , inplace = True)
df.sentiment.replace("negative" , 0 , inplace = True)


C:\Users\wojte\AppData\Local\Temp\ipykernel_26248\3679309076.py:16: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


## Count Vectorized

In [3]:
X = df['review']
Y = df['sentiment']

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=42)

x_train = x_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [4]:
vectorized = CountVectorizer()
x_train_cv= vectorized.fit_transform(x_train)
print(x_train_cv.shape)

(40000, 76074)


In [5]:
from sklearn.linear_model  import LogisticRegression
model = LogisticRegression( max_iter=2000)
model.fit(x_train_cv, y_train)

LogisticRegression(max_iter=2000)

In [6]:
x_test_cv= vectorized.transform(x_test)
y_pred = model.predict(x_test_cv)

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.87      0.86      0.86      5000
           1       0.86      0.87      0.86      5000

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000



In [7]:
def count_acc(y_test, y_pred):
    count = 0
    accurate = 0
    for y_t, y_p in zip(y_test, y_pred):
        if(y_t == y_p):
            accurate += 1
        count += 1
    return accurate/count


In [8]:
count_acc(y_test, y_pred)

0.8643

## tdidf Vectorized - ważenie częstością termów


In [9]:
tf =TfidfVectorizer()
x_train_tfidf = tf.fit_transform(x_train)

In [10]:
x_train_tfidf

<40000x76074 sparse matrix of type '<class 'numpy.float64'>'
	with 3042762 stored elements in Compressed Sparse Row format>

In [11]:
print(x_train_tfidf)

  (0, 16546)	0.15301641197953095
  (0, 74917)	0.10472957402406909
  (0, 54284)	0.053650134453322056
  (0, 26037)	0.09083035885077492
  (0, 51026)	0.1868857855192045
  (0, 16697)	0.15959871523883326
  (0, 73817)	0.06481486145577282
  (0, 52064)	0.07687431511282261
  (0, 36967)	0.06611599380373313
  (0, 1788)	0.059406498378365834
  (0, 19646)	0.18440334196575395
  (0, 17642)	0.10300445929796262
  (0, 64211)	0.10569449360116356
  (0, 22924)	0.10281313931952703
  (0, 38849)	0.04373398023640747
  (0, 50725)	0.0866751079524233
  (0, 28011)	0.15226058980495286
  (0, 22212)	0.10261036159761709
  (0, 40281)	0.061123554142555145
  (0, 61806)	0.14267168444841535
  (0, 28478)	0.15893499423539514
  (0, 27380)	0.09417977757797949
  (0, 39477)	0.1739384072551657
  (0, 29224)	0.13244969139684065
  (0, 33824)	0.09326564577675205
  :	:
  (39999, 10667)	0.1194042211227379
  (39999, 585)	0.1024789144755468
  (39999, 23270)	0.10272353754139776
  (39999, 55778)	0.13895653246719814
  (39999, 31233)	0.1076747

### The tuple represents: (document_id, token_id)
The value following the tuple represents the tf-idf score of a given token in a given document

The tuples that are not there have a tf-idf score of 0

If you want to find what token the token_id corresponds to, check the get_feature_names method.



In [12]:
print(x_train_tfidf.shape)

(40000, 76074)


In [13]:
tf.get_feature_names_out()

array(['aa', 'aaa', 'aaaahhhhhh', ..., 'üvegtigris', 'üzümcü', 'ýs'],
      dtype=object)

In [14]:
tf.get_feature_names_out().shape

(76074,)

In [15]:
tf.get_feature_names_out()[16546]

'definite'

In [16]:
model = LogisticRegression(max_iter=1000)
model.fit(x_train_tfidf, y_train)

LogisticRegression(max_iter=1000)

In [17]:
x_test_tfidf =tf.transform(x_test)

y_pred = model.predict(x_test_tfidf)

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.88      0.86      0.87      5000
           1       0.87      0.89      0.88      5000

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000



In [18]:
count_acc(y_test, y_pred)

0.8758

In [19]:
model.predict(tf.transform(["team varied scully two guy plays bana go arctic research post members died either killing killing discover parasitic point madness problem certain dog lashes anyone could favorite episode season also one favorites arctic environment encloses characters course like lot fun watching characters suddenly start flip dramatic visual effects worm effects skin mind drama characters ends working would usually tension actors bana understand going loved ambiguity highly"]))[0]

1

In [20]:
print(tf.transform(["good film"]))

  (0, 27833)	0.7717709609326816
  (0, 24146)	0.6359006084767062


In [21]:
model.coef_

array([[-0.17145302, -0.10037717, -0.05911979, ..., -0.21562221,
        -0.00469862,  0.02646795]])

In [22]:
model.coef_.shape

(1, 76074)

#### coef_ is of shape (1, n_features) when the given problem is binary. In particular, when multi_class='multinomial', coef_ corresponds to outcome 1 (True) and -coef_ corresponds to outcome 0 (False).

In [23]:
feature_names = tf.get_feature_names_out().tolist()
coef = model.coef_[0]

In [24]:
feature_names.index("definite")

16546

In [25]:
coef[16545]

-0.020579948539877678

In [26]:
def get_coef(words, coef, feature_names):
    output = {}
    for word in words:
        if word in feature_names:
            index = feature_names.index(word)
            c = coef[index]
            output[word] = c
            
    return output

In [27]:
get_coef(["awesome","outstanding","funny","nice" ,"ba", "sad"], coef, feature_names)

{'awesome': 2.8882715430419803,
 'outstanding': 2.115657216019273,
 'funny': 1.037375628934594,
 'nice': 1.5731949294125318,
 'ba': 0.01454259430691023,
 'sad': -0.8065479450395604}

In [28]:
from scipy.sparse import find


def get_coef_sentence(sentence, coef, feature_names, tf):
    sentence = tf.transform([sentence])
    word_indexes = find(sentence)[1]
    output = {}
    for i in word_indexes:
        word = feature_names[i]
        c = coef[i]
        output[word] = "{:.2f}".format(float(c))

    return output

def split_pos_neg(coef_dict):
    pos = {}
    neg = {}
    for key,value in coef_dict.items():
        if float(value) > 0:
            pos[key]=value
        else:
            neg[key] = value
    return pos,neg
        

In [29]:
v = tf.transform(["dsadas film"])

In [30]:
v

<1x76074 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [31]:
print(v[0])

  (0, 24146)	1.0


In [32]:
from scipy.sparse import find

find(v)[1]

array([24146])

In [33]:
i = find(v)[1][0]
i

24146

In [34]:
feature_names[i]

'film'

In [35]:
coef[i]

0.8603875172314974

In [36]:
coef_dict = get_coef_sentence("bad awesome good perfect awfull film outstanding", coef, feature_names,tf)
coef_dict

{'awesome': '2.89',
 'awfull': '-0.19',
 'bad': '-7.92',
 'film': '0.86',
 'good': '2.03',
 'outstanding': '2.12',
 'perfect': '4.92'}

In [37]:
split_pos_neg(coef_dict)

({'awesome': '2.89',
  'film': '0.86',
  'good': '2.03',
  'outstanding': '2.12',
  'perfect': '4.92'},
 {'awfull': '-0.19', 'bad': '-7.92'})

In [13]:
import pickle

In [14]:
filename = 'logistic_regresiom_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [30]:
filename = 'tfidf_model.sav'
pickle.dump(tf, open(filename, 'wb'))

In [31]:
model_tfidf = pickle.load(open('tfidf_model.sav', 'rb'))


In [35]:
a= tf.transform(["team varied scully two guy plays bana go arctic research post members died either killing killing discover parasitic point madness problem certain dog lashes anyone could favorite episode season also one favorites arctic environment encloses characters course like lot fun watching characters suddenly start flip dramatic visual effects worm effects skin mind drama characters ends working would usually tension actors bana understand going loved ambiguity highly"])

In [39]:
type(model.predict(a)[0])

numpy.int64

In [79]:
a = x_test[18]
print(a)

film listed made film available something weird video ed scare films one thing version swv disc earlier version film made wlwt television channel either way film unlike ed intended television broadcast viewing general thus level carnage ratcheted still pretty grim exercise exploitation bloody death purported educational live cincinnati area remember thing shown every year around prom time channel looking one film demonstrates tone uniquely american film last pretty typical extremely heavy whether really affected driving


In [80]:
b= tf.transform([a])
model.predict(b)[0]

1

In [82]:
prob_neg, prob_pos = model.predict_proba(b)[0]

In [83]:
prob_neg

0.3798014832959513